First, import needed modules

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point, LineString, Polygon
import shapely.speedups
from geopy.geocoders import Nominatim
import os

Initalize variables

In [ ]:
path = os.getcwd()
path

Load Dataset which has the endpoints of power line projects

In [ ]:
powerlines = pd.read_csv(f'{path}/data/geodata/powerlines.csv', sep=';')
#powerlines = powerlines.set_index("# BBPlG 2013", drop = False)
powerlines.head()

Note: Int_Start and Int_End indicate whether a point was added to keep the direct line between points in Germany's boundaries
      Short indicates whether lines are < 20 km --> They are later dropped from the analysis

Get the geocoordinates of the 'endpoint' cities using OSM/Nominatim API

In [ ]:
start_locations = geocode(powerlines['Start'], provider='nominatim', user_agent='plm', timeout=4)
start_locations

In [ ]:
end_locations = geocode(powerlines['End'], provider='nominatim', user_agent='plm', timeout=4)
end_locations

Create shapefiles from the start and endpoints dfs

In [ ]:
powerlines_geo_start = start_locations.join(powerlines, how = 'left')
powerlines_geo_start = powerlines_geo_start[['#BBPlG', 'Start', 'geometry', 'New', 'Retro', 'Int_start']]
powerlines_geo_start = powerlines_geo_start.to_crs(epsg=25832)
powerlines_geo_start.to_file(f'{path}/data/geodata/startpoints.shp')
powerlines_geo_start.head()

In [ ]:
powerlines_geo_end = end_locations.join(powerlines, how = 'left')
powerlines_geo_end = powerlines_geo_end[['#BBPlG', 'End', 'geometry', 'New', 'Retro', 'Int_end']]
powerlines_geo_end = powerlines_geo_end.to_crs(epsg=25832) #convert CRS to match BKG data
powerlines_geo_end.to_file(f'{path}/data/geodata/endpoints.shp')
powerlines_geo_end.head()

Create powerlines_geo dataset by merging start and endpoints with the powerlines df

In [ ]:
powerlines_geo = start_locations.join(powerlines, how = 'left')
powerlines_geo.rename(columns = {'geometry':'Start_geometry', 'address':'Start_address'}, inplace = True)
powerlines_geo = end_locations.join(powerlines_geo, how = 'left')
powerlines_geo.rename(columns = {'geometry':'End_geometry', 'address':'End_address'}, inplace = True)
powerlines_geo.head()

Create direct line from start and end coordinates

In [ ]:
try:
    powerlines_geo.insert(loc=0, column='direct_line', value=0)
except Exception:
    pass
for i in range (0, len(powerlines_geo)):
    start = powerlines_geo['Start_geometry'].loc[powerlines_geo.index[i]]
    end = powerlines_geo['End_geometry'].loc[powerlines_geo.index[i]]
    powerlines_geo.loc[powerlines_geo.index[i], 'direct_line'] = LineString([start, end])
powerlines_geo.head()

Export shapefile

In [ ]:
powerlines_geo.rename(columns = {'direct_line':'geometry'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['#BBPlG', 'Start', 'End', 'geometry', 'New', 'Retro', 'Short'])
powerlines_geo = powerlines_geo.set_crs(epsg=4326)
powerlines_geo = powerlines_geo.to_crs(epsg=25832)
powerlines_geo.to_file(f'{path}/data/geodata/powerlines.shp')
powerlines_geo.head()